In [1]:
# Will Hollingsworth, Colton Murray, Alexander Shiveley

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

# Q1

By using the same method in the notes to expand the kernel function for some n and d, m can be found.  After expanding, and simplifying, m = number of terms in the polynomial.

c is set to 1.

To avoid tedious calculations, the expansions were done using sympy.

In [3]:
def get_m(n, d):
    """
    Expands and counts the terms in the polynomial kernel function for some n and d, where c = 1
    """
    # Supports n up to 4
    if n == 1:
        x1, z1 = sp.symbols('x1 z1')
        result = sp.expand((x1*z1+1)**d)
    elif n == 2:
        x1, z1, x2, z2 = sp.symbols('x1 z1 x2 z2')
        result = sp.expand((x1*z1+x2*z2+1)**d)
    elif n == 3:
        x1, z1, x2, z2, x3, z3 = sp.symbols('x1 z1 x2 z2 x3 z3')
        result = sp.expand((x1*z1+x2*z2+x3*z3+1)**d)
    elif n == 4:
        x1, z1, x2, z2, x3, z3, x4, z4 = sp.symbols('x1 z1 x2 z2 x3 z3 x4 z4')
        result = sp.expand((x1*z1+x2*z2+x3*z3+x4*z4+1)**d)    
    return str(result).count('+') + 1

# Find m for n = [1, 4], d = [1, 5]
for n in range(1, 5):
    for d in range(1, 6):
        print('n:', n, '  d:', d, '  m:', get_m(n, d))    

n: 1   d: 1   m: 2
n: 1   d: 2   m: 3
n: 1   d: 3   m: 4
n: 1   d: 4   m: 5
n: 1   d: 5   m: 6
n: 2   d: 1   m: 3
n: 2   d: 2   m: 6
n: 2   d: 3   m: 10
n: 2   d: 4   m: 15
n: 2   d: 5   m: 21
n: 3   d: 1   m: 4
n: 3   d: 2   m: 10
n: 3   d: 3   m: 20
n: 3   d: 4   m: 35
n: 3   d: 5   m: 56
n: 4   d: 1   m: 5
n: 4   d: 2   m: 15
n: 4   d: 3   m: 35
n: 4   d: 4   m: 70
n: 4   d: 5   m: 126


The most noticable pattern is that for some $n_1$, $d_1$, $n_2$, $d_2$ where $n_2 = d_1$ and $d_2 = n_1$, $m_1 = m_2$.
For example for $n = 2$, $d = 3$ and $n = 3$, $d = 2$, results in $m = 10$ for both cases.

This suggests there is some term in the relation like $n + d$ or $n * d$ involved for symmetry.

Another pattern is that for larger $n$ and $d$, $m$ grows large quickly. This suggests operations like exponents or factorials may be involved.

Using the above patterns, the final equation below was found to relate $n$, $d$, and $m$:

$$m = \frac{(n + d)!}{n! d!}$$

# Q2

# Getting the data into Python

In [4]:
# Load the txt as a numpy array of strings, 
# because it includes the column headers
raw_data = np.loadtxt('as6_data.txt', delimiter=' ', dtype=str)

# Convert everything into floats!
clean_data = np.array(raw_data, dtype=float)

clean_data

array([[243.,   3.,  -1.],
       [116., 165.,   1.],
       [198., 127.,  -1.],
       [184., 234.,   1.],
       [165., 231.,   1.],
       [160.,  46.,  -1.],
       [ 70., 169.,   1.],
       [300.,  94.,  -1.],
       [ 95.,  62.,  -1.],
       [ 61., 186.,   1.],
       [ 79., 140.,   1.],
       [245., 185.,  -1.],
       [264.,   2.,  -1.],
       [192., 146.,  -1.],
       [274., 104.,  -1.],
       [170.,  37.,  -1.],
       [172., 144.,  -1.],
       [222., 131.,  -1.],
       [ 35., 204.,   1.],
       [105., 199.,   1.],
       [146.,  40.,  -1.],
       [ 35.,  68.,   1.],
       [221.,  41.,  -1.],
       [ 39.,  16.,  -1.],
       [ 18.,  77.,   1.],
       [ 32., 134.,   1.],
       [ 21.,  61.,   1.],
       [109., 266.,   1.],
       [134., 172.,   1.],
       [100., 183.,   1.],
       [ 45., 158.,   1.],
       [263., 193.,  -1.],
       [173.,  88.,  -1.],
       [299.,  98.,  -1.],
       [ 81., 240.,   1.],
       [ 69., 235.,   1.],
       [152., 189.,   1.],
 

In [10]:
def get_sets(data, split):
    """
    Convenience function that randomly selects a training and test set from the input data.
    
    :param data: (ndarray) the data you want to split
    :param split: (float array) the percentages of the data you want to be TRAINING, VALIDATION, and TESTING data
    
    :returns: (tuple) a tuple where the first element is the training set, and the second element is the test set
    """
    # Randomly shuffle the order from a copy of the data
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    row_count = data.shape[0]

    # calc the number of samples, assumes the input samples are seperated by row
    training_count = round(row_count * split)
    
    training_set = shuffled[:training_count]
    test_set = shuffled[training_count:]
    
    return training_set, test_set

In [11]:
training, test = get_sets(clean_data, .5)

In [12]:
import random

def init_lagrange_multipliers(data):
    # Find positions where the labels are -1, and 1 respectively
    neg = np.where(data[:, 2] == -1)[0]
    pos = np.where(data[:, 2] == 1)[0]
    
    y = data[:, 2]
    alpha = np.array([random.randint(0, 100) for _ in y], dtype=float)
    
    # Calculate the sum of the product of the labels and alpha values
    t = sum(y * np.transpose(alpha))
    
    # If the sum is positive, 
    # then increment a random alpha value corresponding to a negative label
    while t > 0:
        i = np.random.choice(neg)
        alpha[i] += 1
        t = sum(y * np.transpose(alpha))
        
    # If the sum is negative, 
    # then increment a random alpha value corresponding to a positive label
    while t < 0:
        i = np.random.choice(pos)
        alpha[i] += 1
        t = sum(y * np.transpose(alpha))
        
    return alpha

In [13]:
alpha = init_lagrange_multipliers(training)
alpha

array([100.,  63.,  41.,  54.,  61.,  82.,  88.,  57.,  64.,  75.,  64.,
         8.,   6.,  50.,  25.,  77.,  74.,  95.,   9.,  70.,  84.,  98.,
        55.,  86.,  73., 102.,  74.,  82.,  40.,   6.,  19.,  70.,  88.,
        96.,  18.,  82.,  77.,  25.,   5.,  21.,  29.,  40.,  49.,  35.,
        12.,  10.,  75.,  15.,  13.,  97.,  13.,  89.,  67.,  81.,  77.,
        21.,  60.,  10.,  32.,  37.,  80.,  93.,  36.,  93.,  68.,  17.,
        47.,  78.,  54.,  24.,  52.,  21.,  31.,  76.,   2.,  43.,  90.,
        36.,  38.,  42.,  61.,  72.,  46.,  61.,  74.,  74.,  30.,  65.,
        23.,  34.,  15.,  62.,  63.,  56.,  84.,  10.,  33.,  84.,   6.,
        77.,  20.,  51.,  40.,   7., 102., 103.,  55.,  58.,  72.,  48.,
        19.,  71.,  96.,   4.,  51.,  96.,  90.,  12.,  45.,  52.,   7.,
        57.,  84.,  21.,   8.,   5.,  44.,   3.,  94.,  85.,  43.,  91.,
        55.])

In [14]:
sum(training[:, 2] * np.transpose(alpha))

0.0